# Orientation Tuning — Overlay of Filtered Units
Use the controls to filter units; **all matching units are overlaid**.


In [ ]:
import numpy as np, pandas as pd
import ipywidgets as W
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# ---- Load data relative to this notebook (content/) ----
meta  = pd.read_csv("data/meta.csv")                 # columns: unit_id, layer, OSI, selectivity
npz   = np.load("data/tuning_curves.npz")            # arrays: angles (A,), curves (N,A)
angles = npz["angles"]
curves = npz["curves"]
N, A = curves.shape[0], curves.shape[1]

# ---- Controls ----
layer_opts = tuple(sorted(meta["layer"].unique().tolist()))
layers = W.SelectMultiple(options=layer_opts, value=layer_opts, description="Layers", rows=3, layout=W.Layout(width="220px"))
osi    = W.FloatRangeSlider(value=(0.0, 1.0), min=0.0, max=1.0, step=0.01, description="OSI", readout_format=".2f", layout=W.Layout(width="380px"))
sel    = W.FloatRangeSlider(value=(0.0, 1.0), min=0.0, max=1.0, step=0.01, description="Select.", readout_format=".2f", layout=W.Layout(width="380px"))
norm   = W.Dropdown(options=[("Raw","raw"), ("Z-score per unit","z"), ("Peak normalize (max=1)","peak")], value="raw", description="Normalize")
focus  = W.Dropdown(options=[(f"Unit {u}", u) for u in meta["unit_id"].tolist()], value=meta["unit_id"].iloc[0], description="Focus unit")
status = W.HTML()
plot_out = W.Output(layout=W.Layout(border="1px solid #e5e5e5", min_height="420px", width="100%"))

def _normalize(y, mode):
    y = np.asarray(y, float)
    if mode == "raw": return y
    if mode == "z":
        m, s = np.nanmean(y), np.nanstd(y)
        return (y - m) / (s + 1e-12)
    if mode == "peak":
        m = np.nanmax(np.abs(y))
        return y / (m + 1e-12)
    return y

def filtered_idx():
    m = (
        meta["layer"].isin(list(layers.value))
        & (meta["OSI"] >= osi.value[0]) & (meta["OSI"] <= osi.value[1])
        & (meta["selectivity"] >= sel.value[0]) & (meta["selectivity"] <= sel.value[1])
    )
    return np.flatnonzero(m.values)

def update_plot(*_):
    idx = filtered_idx()
    status.value = f"<b>Matched units:</b> {idx.size} | Layers: {', '.join(layers.value)} | OSI: {osi.value[0]:.2f}–{osi.value[1]:.2f} | Select.: {sel.value[0]:.2f}–{sel.value[1]:.2f}"
    with plot_out:
        clear_output(wait=True)
        fig = plt.figure(figsize=(7.5, 4.2), dpi=140)
        ax = plt.gca()
        if idx.size == 0:
            ax.text(0.5, 0.5, "No units match the filter", ha="center", va="center", transform=ax.transAxes)
            ax.set_axis_off()
            plt.show()
            return
        # --- Always overlay all filtered units ---
        for ii in idx:
            y = _normalize(curves[ii, :], norm.value)
            ax.plot(angles, y, lw=1.0, alpha=0.30)
        # --- Focus unit on top (thicker) ---
        if focus.value in meta["unit_id"].values:
            row = meta.index[meta["unit_id"] == focus.value][0]
            y = _normalize(curves[row, :], norm.value)
            ax.plot(angles, y, lw=2.6)
        ax.set_xlabel("Orientation (deg)")
        ax.set_ylabel("Response (a.u.)" if norm.value=="raw" else f"Response ({norm.value})")
        ax.set_title("Orientation tuning (overlay of filtered units)")
        ax.grid(True, alpha=0.25)
        plt.tight_layout()
        plt.show()

# Wire callbacks
for w in (layers, osi, sel, norm, focus):
    w.observe(update_plot, names="value")

# Initial render
update_plot()

controls = W.HBox([W.VBox([layers, norm]), W.VBox([osi, sel, focus])])
display(controls, status, plot_out)
